In [1]:
import pybliometrics

In [2]:
pybliometrics.scopus.utils.create_config()

Creating config file at C:\Users\oguzk\.pybliometrics\config.ini with default paths...
Configuration file successfully created at C:\Users\oguzk\.pybliometrics\config.ini
For details see https://pybliometrics.rtfd.io/en/stable/configuration.html.


In [1]:
import pandas as pd
import numpy as np
import regex as re

from pybliometrics.scopus import AuthorSearch


import pickle

Creating config file at C:\Users\oguzk\.pybliometrics\config.ini with default paths...
Configuration file successfully created at C:\Users\oguzk\.pybliometrics\config.ini
For details see https://pybliometrics.rtfd.io/en/stable/configuration.html.


The main objective of this nb is to create a seperate author_df, to be used with Scopus API Author Search

### Part 1: Creating author_df

In [119]:
with open("q1_date_df", "rb") as fp:
    q1_date_df = pickle.load(fp)

KeyboardInterrupt: 

In [ ]:
q1_date_df.shape

(156754, 20)

In [ ]:
q1_date_df.author.map(type).value_counts()

<class 'list'>     156436
<class 'float'>       318
Name: author, dtype: int64

It should be noted that 249 artcls do not have any author information :(

In [ ]:
q1_author_df = q1_date_df.reset_index().dropna(subset= "author").loc[:,["DOI", "JRNL_ID", "author"]].explode("author")

In [ ]:
q1_author_df.reset_index(drop=True, inplace=True)

#### Expanding "author" column -> aut_df

In [ ]:
aut_df = pd.DataFrame(list(q1_author_df['author']))

In [ ]:
aut_df.affiliation.map(len).map(lambda x: 1 if x>0 else 0).value_counts()

0    530763
1    191411
Name: affiliation, dtype: int64

In [ ]:
aut_df[aut_df.sequence == "first"].affiliation.map(len).map(lambda x: 1 if x>0 else 0).value_counts()

0    116505
1     43729
Name: affiliation, dtype: int64

In [ ]:
# Combining the two datasets:
q1_author_df = pd.concat([q1_author_df, aut_df], axis=1)

In [ ]:
q1_author_df.head()

,DOI,JRNL_ID,author,given,family,sequence,affiliation,ORCID,authenticated-orcid,suffix,name
0,10.1002/aic.14056,1_0,"{'given': 'Phuong-Mai', 'family': 'Nguyen', 's...",Phuong-Mai,Nguyen,first,[],NaN,NaN,NaN,NaN
1,10.1002/aic.14056,1_0,"{'given': 'Audrey', 'family': 'Goujon', 'seque...",Audrey,Goujon,additional,[{'name': 'Institut National de la Recherche A...,NaN,NaN,NaN,NaN
2,10.1002/aic.14056,1_0,"{'given': 'Patrick', 'family': 'Sauvegrain', '...",Patrick,Sauvegrain,additional,[{'name': 'Laboratoire National de métrologie ...,NaN,NaN,NaN,NaN
3,10.1002/aic.14056,1_0,"{'given': 'Olivier', 'family': 'Vitrac', 'sequ...",Olivier,Vitrac,additional,[{'name': 'Institut National de la Recherche A...,NaN,NaN,NaN,NaN
4,10.1002/aic.14601,1_1,"{'given': 'Eiji', 'family': 'Iritani', 'sequen...",Eiji,Iritani,first,[{'name': 'Dept. of Chemical Engineering; Nago...,NaN,NaN,NaN,NaN


In [ ]:
q1_author_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 722174 entries, 0 to 722173
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   DOI                  722174 non-null  object
 1   JRNL_ID              722174 non-null  object
 2   author               722174 non-null  object
 3   given                720608 non-null  object
 4   family               721590 non-null  object
 5   sequence             722174 non-null  object
 6   affiliation          722174 non-null  object
 7   ORCID                51373 non-null   object
 8   authenticated-orcid  51373 non-null   object
 9   suffix               1021 non-null    object
 10  name                 584 non-null     object
dtypes: object(11)
memory usage: 60.6+ MB


In [ ]:
len(q1_author_df[(q1_author_df.ORCID.notna()) & (q1_author_df.affiliation.map(len)==0)])

28055

There are 28K authors that have ORCID info but not any affl. fields.

Can be useful, so will not drop!

In [ ]:
q1_author_df.drop(["suffix", "name", "author"], axis=1, inplace=True)

The initial analysis shows that a .groupby() w/o any preprocessing decreases the total number of authors from 720K to 590K.

However, it is seen that there is a lot of errors & punct. marks in the dataset. These must be cleaned for a better groupby as well as API use.

In [ ]:
# To enable easy & detailed analysis we need to remove NaN values from author_df. However, this will not be necessary for the final q1_date_df, as it should not have any empty auhtor fields.and

q1_author_df.dropna(subset=["family", "given"], inplace=True)

In [ ]:
q1_author_df.drop(["New_given", "New_family",], axis=1, inplace=True)

KeyError: "['New_given', 'New_family'] not found in axis"

The standard for abbr. names will be "A. C."

In [ ]:
# Need to df[0] = df[0].str.strip() first:

q1_author_df.given = q1_author_df.given.str.strip()
q1_author_df.family = q1_author_df.family.str.strip()

In [ ]:
# Remove multiple spaces:
q1_author_df.given = q1_author_df.given.map(lambda x: re.sub(' +', ' ', x))
q1_author_df.family = q1_author_df.family.map(lambda x: re.sub(' +', ' ', x))

In [ ]:
q1_author_df[q1_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.\s[A-Z]\.$", x)))]

,DOI,JRNL_ID,given,family,sequence,affiliation,ORCID,authenticated-orcid,New_given,New_family
121,10.1007/s10441-013-9176-6,1_62,A. S.,Jannot,additional,[],NaN,NaN,a. s.,jannot
125,10.1007/s10441-014-9234-8,1_61,G. P.,Samanta,first,[],NaN,NaN,g. p.,samanta
153,10.1007/s00701-013-1624-1,1_73,M. P.,Meier,first,[],NaN,NaN,m. p.,meier
156,10.1007/s00701-013-1624-1,1_73,M. I.,Ruge,additional,[],NaN,NaN,m. i.,ruge
314,10.1007/s10714-010-0991-7,1_114,J. T.,Firouzjaee,first,[],NaN,NaN,j. t.,firouzjaee
...,...,...,...,...,...,...,...,...,...,...
722047,10.1093/geronb/gbt032,1_24503,K. E.,Cichy,additional,[],NaN,NaN,k. e.,cichy
722048,10.1093/geronb/gbt032,1_24503,K. S.,Birditt,additional,[],NaN,NaN,k. s.,birditt
722057,10.1093/geronb/gbt100,1_24503,E. J.,Nicklett,first,[],NaN,NaN,e. j.,nicklett
722059,10.1093/geronb/gbt100,1_24503,M. S.,Spencer,additional,[],NaN,NaN,m. s.,spencer


In [ ]:
# Case 1: A. C
q1_author_df.loc[q1_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.\s[A-Z]$", x))),"new_given"] = q1_author_df[q1_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.\s[A-Z]$", x)))].given.map(lambda x: x +".")

In [ ]:
# Case 2: A.C
q1_author_df.loc[q1_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.[A-Z]$", x))),"new_given"] = q1_author_df[q1_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.[A-Z]$", x)))].given.map(lambda x: x.replace(".",". ") +".")

In [ ]:
# Case 3: AC
q1_author_df.loc[q1_author_df.given.map(lambda x: bool(re.match("^[A-Z]{2}$", x))),"new_given"] = q1_author_df[q1_author_df.given.map(lambda x: bool(re.match("^[A-Z]{2}$", x)))].given.map(lambda x: x[0] + ". " + x[1] + ".")

In [ ]:
# Case 4: A C
q1_author_df.loc[q1_author_df.given.map(lambda x: bool(re.match("^[A-Z]\s[A-Z]$", x))),"new_given"] = q1_author_df[q1_author_df.given.map(lambda x: bool(re.match("^[A-Z]\s[A-Z]$", x)))].given.map(lambda x: x.replace(" ", ". ") + ".")

In [ ]:
# Case 5: A.C.
q1_author_df.loc[q1_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.[A-Z]\.$", x))), "new_given"] = q1_author_df[q1_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.[A-Z]\.$", x)))].given.map(lambda x: x.replace(".", ". ").strip())

In [ ]:
# Single letter nanmes:
q1_author_df.loc[q1_author_df.given.map(lambda x: bool(re.match("^[A-Z]$", x))),"new_given"] = q1_author_df[q1_author_df.given.map(lambda x: bool(re.match("^[A-Z]$", x)))].given.map(lambda x: x + ".")

In [ ]:
# Single letter nanmes:
q1_author_df[q1_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.$", x)))]

,DOI,JRNL_ID,given,family,sequence,affiliation,ORCID,authenticated-orcid,new_given,new_family
118,10.1007/s10441-013-9176-6,1_62,J.,Demongeot,first,[],NaN,NaN,j.,demongeot
119,10.1007/s10441-013-9176-6,1_62,O.,Hansen,additional,[],NaN,NaN,o.,hansen
120,10.1007/s10441-013-9176-6,1_62,H.,Hessami,additional,[],NaN,NaN,h.,hessami
122,10.1007/s10441-013-9176-6,1_62,J.,Mintsa,additional,[],NaN,NaN,j.,mintsa
123,10.1007/s10441-013-9176-6,1_62,M.,Rachdi,additional,[],NaN,NaN,m.,rachdi
...,...,...,...,...,...,...,...,...,...,...
722041,10.1093/geronb/gbr092,1_24502,M.,Kalmijn,additional,[],NaN,NaN,m.,kalmijn
722043,10.1093/geronb/gbr131,1_24502,M.,Wilson-Genderson,additional,[],NaN,NaN,m.,wilson-genderson
722049,10.1093/geronb/gbt032,1_24503,S.,Zarit,additional,[],NaN,NaN,s.,zarit
722061,10.1093/geronb/gbt121,1_24503,V.,Burholt,first,[],NaN,NaN,v.,burholt


In [ ]:
# Removing [Name], (Name),
# remove "Name" , “Name”,
# remove & % , - ' .
# simplf. ö & ü & � & i

# Main Preprocessing:
def name_simplifier(pdf_read):
    # Lowercase all:
    pdf_modified = pdf_read.lower()
    # Remove everything in brackets & paranthesis:
    pdf_modified = re.sub("[\(\[\{].*?[\)\]\}]", "", pdf_modified)
    # Remove quote names:
    pdf_modified = re.sub('["“].*?["”]', "", pdf_modified)
    # Remove numbers
    pdf_modified = re.sub(r'[0-9]', '', pdf_modified)
    # Remove weird punct.
    pdf_modified = re.sub(r'[&\?\$\+\\\*\^\|]', '', pdf_modified)
    # Simplify acct. a:
    pdf_modified = re.sub(r'[áạàảãăặằẳẵâấậầẩẫ]', 'a', pdf_modified)
    # Simplify acct. i:
    pdf_modified = re.sub(r'[íịìỉĩïǐĭīĩįɨıî]', 'i', pdf_modified)
    # Simplify acct. i:
    pdf_modified = re.sub(r'[éẹèẻẽêếệềểễ]', 'e', pdf_modified)
    # Simplify acct. o:
    pdf_modified = re.sub(r'[óòȯôöǒŏōõǫőốồøṓṑ]', 'o', pdf_modified)
    # Simplify acct. u:
    pdf_modified = re.sub(r'[úùûüǔŭūũů]', 'u', pdf_modified)
    # Remove multiple spaces:
    pdf_modified = re.sub(' +', ' ', pdf_modified)
    # Return pdf_modified
    return pdf_modified



In [ ]:
# Simpli:

q1_author_df.loc[q1_author_df.new_given.isna(),"new_given"] = q1_author_df[q1_author_df.new_given.isna()].given.map(name_simplifier)
q1_author_df.loc[:,"new_family"] = q1_author_df.family.map(name_simplifier)

In [ ]:
# After process stripping:

q1_author_df.given = q1_author_df.given.str.strip()
q1_author_df.family = q1_author_df.family.str.strip()

In [ ]:
q1_author_df[["given","family","new_given","new_family"]].sample(20)

,given,family,new_given,new_family
662758,Juan,Du,juan,du
112153,Fabiola H.,Gerpott,fabiola h.,gerpott
378415,Nina,Hofmann,nina,hofmann
503665,Steven,Jay,steven,jay
272638,Srdjan,Pusara,srdjan,pusara
380487,A. R.,Mermut,a. r.,mermut
98232,Shiun-Yuan,Hsu,shiun-yuan,hsu
713101,Marta,Lachowska,marta,lachowska
684415,Per,Gardeström,per,gardestrom
454476,John E.,Savage,john e.,savage


In [ ]:
q1_author_df.groupby(["new_given","new_family"])["DOI"].count()

new_given   new_family  
                            1
            rahmatd         1
            santiano        1
 jing       cai             1
 kim        keamy           1
                           ..
о.о.        dan’kiv         1
т.а.        denisova        1
тatiana а.  grebennikova    1
ᴌukasz      boguszewicz     1
�lvaro      petracco        1
Name: DOI, Length: 585967, dtype: int64

### Part 2: Adding Subject Areas

In [ ]:
with open("q1_jrnl_df", "rb") as fp:
    q1_jrnl_df = pickle.load(fp)

In [ ]:
q1_author_df = pd.merge(left=q1_author_df, right=q1_jrnl_df["SA_list"], how="left", left_on="JRNL_ID", right_index=True)

### Part 3: Scopus API Trials

In [ ]:
# copied from 6a:

def Scopus_author(author_list):
    for aut in author_list:
        name = aut["given"]
        surname = aut["family"]
        s = AuthorSearch(f'AUTHLAST({surname}) and AUTHFIRST({name})')
        if s.get_results_size()>0:
            aut["country"] = s.authors[0]._asdict()["country"]
        else:
            aut["country"] = None
    return author_list

In [ ]:
# copied from 6a:

# After adding the country field for every author in "author" column:
# Can use to create a seperate "author_countries" column

def country_counter(author_col):
    return Counter([aut["country"] for aut in author_col])


Useful stuff:

In [17]:
scopus_codes_dict = {'Agricultural and Biological Sciences': 'AGRI',
 'Arts and Humanities': 'ARTS',
 'Biochemistry, Genetics and Molecular Biology': 'BIOC',
 'Business, Management and Accounting': 'BUSI',
 'Chemical Engineering': 'CENG',
 'Chemistry': 'CHEM',
 'Computer Science': 'COMP',
 'Decision Sciences': 'DECI',
 'Dentistry': 'DENT',
 'Earth and Planetary Sciences': 'EART',
 'Economics, Econometrics and Finance': 'ECON',
 'Energy': 'ENER',
 'Engineering': 'ENGI',
 'Environmental Science': 'ENVI',
 'Health Professions': 'HEAL',
 'Immunology and Microbiology': 'IMMU',
 'Materials Science': 'MATE',
 'Mathematics': 'MATH',
 'Medicine': 'MEDI',
 'Neuroscience': 'NEUR',
 'Nursing': 'NURS',
 'Pharmacology, Toxicology and Pharmaceutics': 'PHAR',
 'Physics and Astronomy': 'PHYS',
 'Psychology': 'PSYC',
 'Social Sciences': 'SOCI',
 'Veterinary': 'VETE',
 'Multidisciplinary': 'MULT'}

def sbj_area_query_creator(sbj_list):
    query = " AND ".join([f"SUBJAREA({scopus_codes_dict[sbj]})" for sbj in sbj_list])
    return query

In [ ]:
testo = q1_author_df.sample(1)

testo

,DOI,JRNL_ID,given,family,sequence,affiliation,ORCID,authenticated-orcid,new_given,new_family,SA_list
128084,10.1021/acs.jmedchem.6b00176,1_4104,Vanessa,Rada,additional,[],NaN,NaN,vanessa,rada,"[Pharmacology, Toxicology and Pharmaceutics, B..."


In [ ]:
Scopus_author(testo)

['United States']

In [ ]:
# ScopusQueryError: Found 6574 matches.  The query fails to return more than 5000 entries.  Change your query such that it returns fewer entries.

In [129]:
# VERSION 1 - To be used w/ .apply()

def Scopus_author(row):
    name = row["new_given"]
    surname = row["new_family"]
    sa_list = sbj_area_query_creator(row.SA_list)
    try:
        s = AuthorSearch(f'AUTHLAST({surname}) AND AUTHFIRST({name}) AND ({sa_list})')
    except:
        return None
    if s.get_results_size() > 0:
        return [s.authors[i]._asdict()["country"] for i in range(s.get_results_size())]
    else:
        return np.nan

In [ ]:
# VERSION 2 - To be used w/ .samle()

def Scopus_author_sample(row):
    name = row["new_given"].item()
    surname = row["new_family"].item()
    sa_list = sbj_area_query_creator(row.SA_list.item())
    try:
        s = AuthorSearch(f'AUTHLAST({surname}) AND AUTHFIRST({name}) AND ({sa_list})')
    except:
        return None
    if s.get_results_size() > 0:
        return [s.authors[i]._asdict()["country"] for i in range(s.get_results_size())]
    else:
        return np.nan

In [124]:
testo_df = q1_author_df.sample(6000)

In [130]:
testo_df["scopus_result"] = testo_df.apply(Scopus_author, axis=1)

In [135]:
testo_df.scopus_result.map(type).value_counts()

<class 'NoneType'>    3983
<class 'list'>        1974
<class 'float'>         43
Name: scopus_result, dtype: int64

In [143]:
with open("testo_df","wb") as p:
    pickle.dump(testo_df, p)

In [ ]:
with open("q1_author_df","wb") as p:
    pickle.dump(q1_author_df, p)

In [140]:
print(s_s)

Search 'AUTHLAST(Evans) and AUTHFIRST(David G.) AND (SUBJAREA(MEDI) OR SUBJAREA(BIOC) OR SUBJAREA(NEUR))' yielded 15 authors as of 2022-03-22:
    Evans, D. Gareth R.; AUTHOR_ID:6505764222 (1,084 document(s))
    Evans, David Geraint; AUTHOR_ID:57199095509 (450 document(s))
    Evans, David G.; AUTHOR_ID:34770687200 (22 document(s))
    Evans, David G.; AUTHOR_ID:57223817001 (10 document(s))
    Evans, David G.; AUTHOR_ID:55478754500 (9 document(s))
    Evans, David G.; AUTHOR_ID:55478764200 (7 document(s))
    Evans, David G.; AUTHOR_ID:57209538123 (5 document(s))
    Evans, David G.; AUTHOR_ID:57214022971 (3 document(s))
    Evans, David G.; AUTHOR_ID:55568514534 (3 document(s))
    Evans, William David George; AUTHOR_ID:55554688500 (3 document(s))
    Evans, David G.; AUTHOR_ID:55478772600 (2 document(s))
    Evans, David Glyn; AUTHOR_ID:57214023150 (1 document(s))
    Evans, David G.; AUTHOR_ID:55478811400 (1 document(s))
    Evans, David G.; AUTHOR_ID:55478794700 (1 document(s))
 

### Part 4: Best way to use Scopus API

In [1]:
# import pybliometrics

# pybliometrics.scopus.utils.create_config()

import pandas as pd
import numpy as np
import regex as re

from pybliometrics.scopus import AuthorSearch


import pickle

In [2]:
scopus_codes_dict = {'Agricultural and Biological Sciences': 'AGRI',
 'Arts and Humanities': 'ARTS',
 'Biochemistry, Genetics and Molecular Biology': 'BIOC',
 'Business, Management and Accounting': 'BUSI',
 'Chemical Engineering': 'CENG',
 'Chemistry': 'CHEM',
 'Computer Science': 'COMP',
 'Decision Sciences': 'DECI',
 'Dentistry': 'DENT',
 'Earth and Planetary Sciences': 'EART',
 'Economics, Econometrics and Finance': 'ECON',
 'Energy': 'ENER',
 'Engineering': 'ENGI',
 'Environmental Science': 'ENVI',
 'Health Professions': 'HEAL',
 'Immunology and Microbiology': 'IMMU',
 'Materials Science': 'MATE',
 'Mathematics': 'MATH',
 'Medicine': 'MEDI',
 'Neuroscience': 'NEUR',
 'Nursing': 'NURS',
 'Pharmacology, Toxicology and Pharmaceutics': 'PHAR',
 'Physics and Astronomy': 'PHYS',
 'Psychology': 'PSYC',
 'Social Sciences': 'SOCI',
 'Veterinary': 'VETE',
 'Multidisciplinary': 'MULT'}

def sbj_area_query_creator(sbj_list):
    query = " AND ".join([f"SUBJAREA({scopus_codes_dict[sbj]})" for sbj in sbj_list])
    return query

In [3]:
with open("q1_author_df", "rb") as fp:
    q1_author_df = pickle.load(fp)

In [4]:
author_records = q1_author_df[["new_given","new_family","SA_list"]].sample(10).to_dict("records")

#### 4.1 - w/o any API Keys 

In [19]:
for author in author_records[:10]:
    name = author["new_given"]
    surname = author["new_family"]
    sa_list = sbj_area_query_creator(author["SA_list"])

    s = AuthorSearch(f'AUTHLAST({surname}) AND AUTHFIRST({name}) AND ({sa_list})')
    author["affil"] = [s.authors[i]._asdict()["country"] for i in range(s.get_results_size())]


Scopus401Error: Invalid API Key

__FAIL__ - We can NOT get any info w/o API keys.

#### 4.2 - w/ already used(?) API key

In [5]:
for author in author_records[:10]:
    name = author["new_given"]
    surname = author["new_family"]
    sa_list = sbj_area_query_creator(author["SA_list"])

    s = AuthorSearch(f'AUTHLAST({surname}) AND AUTHFIRST({name}) AND ({sa_list})')
    author["affil"] = [s.authors[i]._asdict()["country"] for i in range(s.get_results_size())]


Scopus429Error: 

__FAIL__ - Already used API key gives the 429 error as expected & can NOT be used until the following week!

#### 4.3 - w/o VPN connection

In [6]:
for author in author_records:
    name = author["new_given"]
    surname = author["new_family"]
    sa_list = sbj_area_query_creator(author["SA_list"])
    

    s = AuthorSearch(f'AUTHLAST({surname}) AND AUTHFIRST({name}) AND ({sa_list})', download=False)
    n = s.get_results_size()

    if n > 5000:
        author["affil"] = "Too many results!"
    else:
        author["affil"] = [s.authors[i]._asdict()["country"] for i in range(s.get_results_size())]

Scopus401Error: The requestor is not authorized to access the requested view or fields of the resource

__FAIL__ - does NOT work w/o VPN connection!

#### 4.4 - w/ download = False 

Is it reasonable to use download = False to check result_size & use the same request to also get the info needed

In [5]:
for author in author_records:
    name = author["new_given"]
    surname = author["new_family"]
    sa_list = sbj_area_query_creator(author["SA_list"])
    

    s = AuthorSearch(f'AUTHLAST({surname}) AND AUTHFIRST({name}) AND ({sa_list})', download=False)
    n = s.get_results_size()

    if n > 5000:
        author["affil"] = "Too many results!"
    else:
        author["affil"] = [s.authors[i]._asdict()["country"] for i in range(s.get_results_size())]

TypeError: 'NoneType' object is not subscriptable

In [6]:
s.get_key_remaining_quota()

'5000'

__FAIL__ - If download = False, can not use the query result! 

__HOWEVER__, the remaining quota is still 5000, so probab. download = False does not use quota!!

#### 4.5 - Combining all:

In [7]:
for author in author_records:
    name = author["new_given"]
    surname = author["new_family"]
    sa_list = sbj_area_query_creator(author["SA_list"])
    

    n = AuthorSearch(f'AUTHLAST({surname}) AND AUTHFIRST({name}) AND ({sa_list})', download=False).get_results_size()

    if n > 5000:
        author["affil"] = "Too many results!"
    else:
        s = AuthorSearch(f'AUTHLAST({surname}) AND AUTHFIRST({name}) AND ({sa_list})')
        author["affil"] = [s.authors[i]._asdict()["country"] for i in range(n)]

In [8]:
s.get_key_remaining_quota()

'4970'

In [10]:
pd.DataFrame(author_records)

,new_given,new_family,SA_list,affil
0,yichuan,xiao,"[Biochemistry, Genetics and Molecular Biology]","[China, China, China, China]"
1,ahmed ibrahim fadhil,al-adly,"[Energy, Chemistry, Chemical Engineering]",[Iraq]
2,alex,nicol-harper,[Agricultural and Biological Sciences],[United Kingdom]
3,konrad,gajewski,"[Social Sciences, Arts and Humanities, Agricul...",[Canada]
4,ajmal,ahmad,"[Pharmacology, Toxicology and Pharmaceutics]","[Saudi Arabia, Saudi Arabia]"
5,megan,stolen,"[Chemistry, Environmental Science, Medicine]",[United States]
6,dragana,životić,[Earth and Planetary Sciences],[Serbia]
7,yotam,levy,"[Neuroscience, Medicine]",[United Kingdom]
8,michael a. e.,andersen,"[Engineering, Energy]",[Denmark]
9,christopher j.,grim,"[Immunology and Microbiology, Medicine]",[United States]


It is WORKING!!!! However, the quota is now 4970 intead of the expected number 4990. Need to check this issue!